In [3]:
import matplotlib.pyplot as plt # plotting library
import numpy as np # this module is useful to work with numerical arrays
import pandas as pd # this module is useful to work with tabular data
import random # this module will be used to select random samples from a collection
import os # this module will be used just to create directories in the local filesystem
from tqdm.notebook import tqdm # this module is useful to plot progress bars

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn

## Training function

In [ ]:
### Define the loss function
loss_fn = torch.nn.MSELoss()

params_to_optimize = [
    {'params': encoder.parameters()},
    {'params': decoder.parameters()}
]

# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

# Move both the encoder and the decoder to the selected device
encoder.to(device)
decoder.to(device)

### LR Scheduler

In [4]:
from torch.optim import Adam
from torch.optim.lr_scheduler import LinearLR, ExponentialLR

### Define an optimizer (both for the encoder and the decoder!)
lr = 3e-4 # Learning rate

#function for the custom learning rate:
def custom_lr(epoch, lr = 3e-4):

    if epoch < 8: # linear increase
        return lr + 1.125e-4 * epoch
    if (epoch >= 8 and epoch < 16): # linear decrease
        return lr*10 - 1.125e-4 * (epoch - 8 )
    if epoch >= 16: # linear decrease
        return lr - 7.4825e-5 * (epoch - 16)

optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda=custom_lr)


NameError: name 'params_to_optimize' is not defined

In [ ]:
### Training function
def train_epoch(encoder, decoder, device, dataloader, loss_fn, optimizer):
    # Set train mode for both the encoder and the decoder
    encoder.train()
    decoder.train()

    # abbiamo già definito l'optimizer nella cella precedente
    
    losses = []
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for batch, _ in dataloader: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        
        batch = batch.to(device)
        #encode
        y_encoder_pred = encoder(batch)
        #decode
        y_decoder_pred = decoder(y_encoder_pred)

        loss = loss_fn(y_decoder_pred, batch) # funziona anche per le matrici?

        optim.zero_grad()
        loss.backward()
        optim.step()

        # scheduler.step() # in teoria questo va alla fine di una epoca
        
        losses.append(loss.detach().cpu().numpy())
    
    
    losses = np.mean(losses)
    return losses

In [ ]:
### Training cycle
num_epochs = 20
for epoch in range(num_epochs):
    print('EPOCH %d/%d' % (epoch + 1, num_epochs))
    ### Training (use the training function)
    train_loss = train_epoch(
        encoder=encoder, 
        decoder=decoder, 
        device=device, 
        dataloader=train_dataloader, 
        loss_fn=loss_fn, 
        optimizer=optim)
    print(f'TRAIN - EPOCH {epoch+1}/{num_epochs} - loss: {train_loss}')

    ### Validation  (use the testing function)
    val_loss = test_epoch(
        encoder=encoder, 
        decoder=decoder, 
        device=device, 
        dataloader=test_dataloader, 
        loss_fn=loss_fn)
    # Print Validationloss
    print(f'VALIDATION - EPOCH {epoch+1}/{num_epochs} - loss: {val_loss}\n')

    ### step of the lr scheduler
    scheduler.step() # alla fine di ogni epoca

    ### Plot progress